EXEMPLUL Boston housing - LASSO

In [65]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import scale 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

In [66]:
df = pd.read_csv("boston_housing.csv", header = 0)

In [67]:
df=df.dropna(how='any')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 401 entries, 0 to 449
Data columns (total 14 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     401 non-null    float64
 1   ZN       401 non-null    float64
 2   INDUS    401 non-null    float64
 3   CHAS     401 non-null    float64
 4   NOX      401 non-null    float64
 5   RM       401 non-null    float64
 6   age      401 non-null    float64
 7   DIS      401 non-null    float64
 8   RAD      401 non-null    int64  
 9   IMPOZIT  401 non-null    float64
 10  PTRATIO  401 non-null    float64
 11  B        401 non-null    float64
 12  LSTAT%   401 non-null    float64
 13  MEDV     401 non-null    float64
dtypes: float64(13), int64(1)
memory usage: 47.0 KB


In [68]:
y = df['MEDV']
X = df[[c for c in df.columns if c != 'MEDV']]

X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2)


In [69]:
X_train.columns

Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'age', 'DIS', 'RAD',
       'IMPOZIT', 'PTRATIO', 'B', 'LSTAT%'],
      dtype='object')

In [70]:
X_test.columns

Index(['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'age', 'DIS', 'RAD',
       'IMPOZIT', 'PTRATIO', 'B', 'LSTAT%'],
      dtype='object')

In [82]:
lambda_values = 10**np.linspace(10,-3,100)*0.5

print(min(lambda_values),max(lambda_values))

0.0005 5000000000.0


In [83]:
def std_input(some_input, means,stds):
    
    return (some_input - means)/stds

In [84]:
std_input(X_test, np.mean(X_test),np.std(X_test))

,CRIM,ZN,INDUS,CHAS,NOX,RM,age,DIS,RAD,IMPOZIT,PTRATIO,B,LSTAT%
111,-0.535472,-0.511633,-0.011939,-0.282843,0.067200,0.570671,0.476963,-0.625125,-0.275250,0.368719,-0.058900,0.378751,-0.259937
91,-0.556616,-0.511633,-1.013874,-0.282843,-0.471269,0.044159,0.212441,-0.403522,-0.793745,-0.708619,-0.058900,0.353168,-0.576916
29,-0.225581,-0.511633,-0.295820,-0.282843,-0.016356,0.501036,0.672778,0.209494,-0.534498,-0.462561,1.354700,0.186127,0.034401
396,1.448138,-0.511633,1.216191,-0.282843,1.422655,0.044159,0.971654,-1.159998,2.057977,1.924873,1.001300,0.395179,1.229544
184,-0.541576,-0.511633,-1.158092,-0.282843,-0.480553,-1.316281,0.758662,-0.459217,-0.664122,-1.220687,-0.058900,0.321189,0.357850
...,...,...,...,...,...,...,...,...,...,...,...,...,...
428,1.962002,-0.511633,1.216191,-0.282843,1.292680,-0.315907,0.356726,-1.021670,2.057977,1.924873,1.001300,-3.369130,1.577251
187,-0.543064,1.542223,-1.009319,-0.282843,-0.954034,0.684466,-0.914356,-0.031244,-0.404874,0.142611,-1.207450,0.357181,-0.822737
60,-0.518809,0.629398,-0.752763,-0.282843,-0.805491,-1.083596,-0.052082,1.805719,-0.016003,-0.615516,0.780425,0.372731,0.223619
105,-0.524549,-0.511633,-0.232061,-0.282843,-0.183467,-0.896769,0.995702,-0.930110,-0.404874,0.049507,1.310525,0.359438,0.760543


In [85]:
# 1. eliminare X_test, y_test din date
#- done

In [110]:
#Pasii 2-5 -> LassoCV
lambda_values = 10**np.linspace(10,-3,100)*0.5
# normalize = True pentru ca toti predictorii sa fie normalizati la aceeiasi scala
lassocv = LassoCV(alphas = lambda_values, cv = 10, max_iter = 100000, normalize = True)
lassocv.fit(X_train, y_train)

#pas 5
lambda_optim = lassocv.alpha_

# pas 6: reantrenare pe toate datele de antrenament
lasso = Lasso(max_iter = 100000, normalize = True)
lasso.set_params(alpha=lambda_optim)
lasso.fit(X_train, y_train)

# pas 7 - raportatea erorii
cv_alpha_train_error = mean_squared_error(y_train, lasso.predict(X_train))

print("Eroare medie patratica (MSE) pe date antrenatemant", cv_alpha_train_error)

cv_alpha_test_error = mean_squared_error(y_test, 
                                         lasso.predict(X_test))

print("MSE date test :", cv_alpha_test_error)

print('Diferenta procentuala dintre MSE pe test si pe date antrenament:  = {0}% '.format( 
      round(100*abs((cv_alpha_train_error-cv_alpha_test_error)/cv_alpha_train_error),2)))
            
# score(X, y[, sample_weight]) - coeficientul de determinare R^2 all predictiei.
print("R^2 pe date de anternament: {0}".format(lasso.score(X_train,y_train)))
print("R^2 pe date de test: {0}".format(lasso.score(X_test,y_test)))
            

Eroare medie patratica (MSE) pe date antrenatemant 20.294451579036753
MSE date test : 25.99704679696588
Diferenta procentuala dintre MSE pe test si pe date antrenament:  = 28.1% 
R^2 pe date de anternament: 0.7454562880170414
R^2 pe date de test: 0.695765265414354


Să vedem căți predictori a suprimat LASSO

In [108]:

df_coef = pd.DataFrame(lasso.coef_, index = X.columns).reset_index().rename(columns={'index':'Predictor',0:'Coeficient'})
print("LASSO a ales " + str(sum(df_coef.Coeficient != 0)) + " predictor(i) și a eliminat " +  \
      str(sum(df_coef.Coeficient == 0)) + " predictor(i)")

LASSO a ales 12 predictor(i) și a eliminat 1 predictor(i)


Grafic cu importanta vectorilor:

In [105]:
coef

,Predictor,Coeficient
0,CRIM,-0.000000
1,ZN,0.017936
2,INDUS,0.012289
3,CHAS,1.868577
4,NOX,-4.951996
5,RM,5.615165
6,age,0.002623
7,DIS,-0.912121
8,RAD,0.165191
9,IMPOZIT,-0.008774


În mărime absolută, și in condițiile în care predictorii au fost standardizați (deci toți iau valori în aceeși scală, comparabile), coeficienți determină importanța predictorilor - coeficient mare => importanță mare:

In [102]:
import plotly.graph_objects as go

fig = go.Figure(go.Bar(
            x=abs(coef).sort_values(),
            y=abs(coef).sort_values().index,
            orientation='h'))

fig.update_layout( title = "Importanța Coefficienților în modelul Lasso")
fig.show()